In [1]:
import dask_awkward as dak
import awkward as ak
import pandas as pd

from distributed import Client
import glob
import os

client =  Client(n_workers=40,  threads_per_worker=1, processes=True, memory_limit='8 GiB')

In [2]:
from modules import selection_debug

In [3]:
V1_fields_2compute = [
    # "gjj_mass",
    "wgt_nominal",
    "nBtagLoose_nominal",
    "nBtagMedium_nominal",
    "mu1_pt",
    "mu2_pt",
    "mu1_eta",
    "mu2_eta",
    "mu1_phi",
    "mu2_phi",
    "dimuon_pt",
    "dimuon_eta",
    "dimuon_phi",
    "dimuon_mass",
    "jet1_phi_nominal",
    "jet1_pt_nominal",
    "jet2_pt_nominal",
    "jet2_phi_nominal",
    "jet1_eta_nominal",
    "jet2_eta_nominal",
    "jj_mass_nominal",
    "jj_dEta_nominal",
    # "region",
    "event",
    "njets_nominal",
    # "run",
    # "event",
    # "luminosityBlock",
    "nfatJets_drmuon",
    "MET_pt"
]

In [4]:
# category="ggh"
category="vbf"
# region="signal"
# region="h-peak"
region="h-sidebands"


In [9]:
def getYield(process, load_path, do_vbf_filter_study=False, year="-"):
    print("{line} {process} {line}".format(line="="*5, process=process))

    fields = V1_fields_2compute
    if "data" in process and "gjj_mass" in fields:
        fields.remove("gjj_mass")
    if "data" not in process:
        fields.append("gjj_mass")

    filelist = glob.glob(f"{load_path}/{process}")
    total_integral = 0
    for file in filelist:
        # print(f"\tfile: {file}")
        events_data = dak.from_parquet(f"{file}/*/*.parquet")
        print(events_data.fields)
        events_data = ak.zip({field: events_data[field] for field in fields}).compute()
        events_data = selection_debug.applyRegionCatCuts(events_data, category=category, region_name=region, process=process, variation="nominal", do_vbf_filter_study=do_vbf_filter_study)

        wgts = ak.fill_none(events_data.wgt_nominal, value=1.0)
        data_yield = ak.sum(wgts)
        # print(f"\tSum weights: {data_yield}")
        df = pd.DataFrame({field: ak.fill_none(events_data[field], value=-999.9) for field in events_data.fields})
        print(f"\tdata_yield for {file.split("/")[-1]}: {data_yield}")
        total_integral += data_yield
    print(f"\t==> Total Yield: {total_integral:.3f}")
    return process,region,year,total_integral

In [10]:
import csv
outfile = "yield_MC_Default.csv"
years = ["2018", "2017", "2016preVFP", "2016postVFP"]

# years = ["2018"]
info = []
for year in years:
    print("{vspace}{lspace} {year} {lspace}".format(vspace="\n\n", lspace="*"*5, year=year))
    # load_path = f"/depot/cms/users/shar1172/hmm/copperheadV1clean/Run2_nanoAODv12_UpdatedQGL_FixPUJetIDWgt/stage1_output/{year}/compacted"
    # load_path = f"/depot/cms/hmm/shar1172/hmm_ntuples/copperheadV1clean/Run2_nanoAODv12_AK8jets/stage1_output/{year}/f1_0"
    load_path = f"/depot/cms/hmm/shar1172/hmm_ntuples/copperheadV1clean/Run2_nanoAODv12_AK8jets/stage1_output/{year}/f1_0"
    # info.append(
    #     getYield(process = "data*", load_path = load_path, year=year, do_vbf_filter_study=False)
    # )
    info.append(
        getYield(process = "vbf_powheg_dipole", load_path = load_path, year=year, do_vbf_filter_study=False)
    )
    # info.append(
    #     getYield(process = "ggh_powhegPS", load_path = load_path, year=year, do_vbf_filter_study=False)
    # )
    # info.append(
    #     getYield(process = "dy_VBF_filter", load_path = load_path, year=year, do_vbf_filter_study=True)
    # )
    # info.append(
    #     getYield(process = "dy_M*_MiNNLO", load_path = load_path, year=year, do_vbf_filter_study=True)
    # )    
    # info.append(
    #     getYield(process = "ewk_lljj_mll50_mjj120", load_path = load_path, year=year, do_vbf_filter_study=False)
    # )   
    # info.append(
    #     getYield(process = "ttjets_*", load_path = load_path, year=year, do_vbf_filter_study=False)
    # )       
    # getYield("st_*")
    # getYield("ww_2l2nu")
    # getYield("wz_1l1nu2q")
    # getYield("wz_2l2q")
    # getYield("wz_3lnu")
    # getYield("zz")
    # getYield("www")
    # getYield("wwz")
    # getYield("wzz")
    # getYield("zzz")

with open(outfile, 'w', newline='') as out_f:
    writer = csv.writer(out_f)
    writer.writerow(['sample', 'region', 'year', 'yield'])
    writer.writerows(info)



***** 2018 *****
===== vbf_powheg_dipole =====
['event', 'PV_npvs', 'PV_npvsGood', 'MET_pt', 'MET_phi', 'MET_sumEt', 'mu1_pt', 'mu1_ptErr', 'mu2_pt', 'mu2_ptErr', 'mu1_pt_over_mass', 'mu2_pt_over_mass', 'mu1_eta', 'mu2_eta', 'mu1_phi', 'mu2_phi', 'mu1_charge', 'mu2_charge', 'mu1_iso', 'mu2_iso', 'mu1_pt_over_mu2_pt', 'mu1_eta_over_mu2_eta', 'nmuons', 'dimuon_mass', 'dimuon_pt', 'dimuon_pt_log', 'dimuon_eta', 'dimuon_rapidity', 'dimuon_phi', 'dimuon_dEta', 'dimuon_dPhi', 'dimuon_dR', 'acoplanarity', 'dimuon_ebe_mass_res', 'dimuon_ebe_mass_res_rel', 'uncalibrated_dimuon_ebe_mass_res', 'dimuon_cos_theta_cs', 'dimuon_phi_cs', 'dimuon_cos_theta_eta', 'dimuon_phi_eta', 'dimuon_pt_over_MET_pt', 'dimuon_pt_over_jet1_pt', 'dimuon_pt_over_jet2_pt', 'mu1_pt_raw', 'mu2_pt_raw', 'mu1_pt_fsr', 'mu2_pt_fsr', 'year', 'run', 'luminosityBlock', 'fraction', 'jet1_default_pt_nominal', 'jet1_default_eta_nominal', 'jet1_default_phi_nominal', 'jet1_default_mass_nominal', 'jet2_default_pt_nominal', 'jet2_de